# 正規分布

前節では正規分布ではない場合について触れただ，やはり統計では正規分布が基本となる．なにしろ中心極限定理により，母集団の分布がどうであれ標本の平均は正規分布になることが決まっているのだ．ここでは正規分布の扱いについて詳しく見ていく．

分布が正規分布のとき，平均値からどのぐらい離れたスコアがどのぐらいの確率で出現するかはよく知られている．
平均値から標準偏差1個分の領域には全体の 68% が含まれる．$2\sigma$以内には96%，$3\sigma$以内にはほぼ全て (99.7%) となる．
念のため確かめてみると

In [2]:
import random

mu = 10
sigma = 1

Ns = 100000
cat = [0,0,0,0,0,0,0]

for s in range(Ns):
    x = random.gauss(mu, sigma)
    for c in range(7):
        if (x < mu + sigma * (c-3) ):
            cat[c] += 1
            break

for c in range(7):
    print("< %dsigma: %d (%.3f%%)" % (c-3, cat[c], cat[c]/Ns * 100) )
    
print("> +3sigma: %d (%.3f%%)" % (Ns - sum(cat), (Ns - sum(cat))/Ns * 100) )

< -3sigma: 141 (0.141%)
< -2sigma: 2100 (2.100%)
< -1sigma: 13554 (13.554%)
< 0sigma: 34239 (34.239%)
< 1sigma: 34029 (34.029%)
< 2sigma: 13653 (13.653%)
< 3sigma: 2140 (2.140%)
> +3sigma: 144 (0.144%)


サンプル数が十分に大きければ当然ながら理論通りになる．平均値$\mu$や標準偏差$\sigma$を変えてもこの数字は変わらない．

## Zスコア

ということは，ある標本がどのぐらいレアなものなのかはスコアそのものではなく，平均値からの距離で評価するのが正しい．
平均値からの距離を標準偏差で正規化した数値がZスコアで，
$$Z = \frac{X - \mu}{\sigma}$$
で定義される．Zスコアの平均値は0，標準偏差は1だ．通常，Zスコアは $-3 < Z < +3$ の範囲にある．この範囲を超える確率はわずか 0.3%だ．
(しかしトランジスタを10億個ほど集積すると，300万個も$3\sigma$を超えてしまう．困ったものだ．)

### 偏差値 (Tスコア)

Zスコアを平均50，標準偏差10になるように補正したものが偏差値 (Tスコア) である．つまり偏差値80はZスコア +3 に相当するのでそうそう出現するものではない．なお偏差値は100を超えることもあれば負の値になることもある．自分のスコアと，自分以外のスコアの平均と標準偏差が与えられれば偏差値はすぐに計算できる．


In [42]:
import numpy as np

score = 0
mu = 90
sigma = 15
Ns = 10000

samples = np.random.normal(loc=mu, scale=sigma, size=Ns)
samples = np.append(samples, score)

z = (score - samples.mean()) / samples.std()

print( "Z-score: %.2f, T-score: %.1f" % (z, z*10 + 50) )

Z-score: -5.98, T-score: -9.8


## パーセンタイルランク

パーセンタイルランクとは，そのスコアより下に標本群全体の何%が入るかを表す．値は0から100，スコアが平均値ならパーセンタイルランクは50になる．
言うまでもなくZスコアとパーセンタイルは対応するため変換表が作成可能である．

|Zスコア|パーセンタイルランク|
|----|----|
|-3|0.15|
|-2|2|
|-1|16|
|0|50|
|1|84|
|2|98|
|3|99.85|

numpy は配列から条件を満たす要素を抽出する方法が提供されているので，パーセンタイルランクを計算するのも簡単である．

In [26]:
import numpy as np

score = 70

Ns=100
mu=70
sigma=10

samples = np.random.normal(loc=mu, scale=sigma, size=Ns)

print(samples[samples < score].size / Ns * 100)

53.0


## 正規分布の応用 : Gott の原理

正規分布の応用は無数にある (というより統計で正規分布を使っていないものの方が珍しい)．ここで紹介する Gott の原理とは，
「ある現象があとどのぐらい続くか」を予測する手法である．

**Gottの原理** : ある現象が $t_{past}$ だけ続いているとすると，その現象はあと$t_{past}/n$から$n\times t_{past}$ぐらい続く

$n$は信頼区間に応じたパラメータで，信頼区間50% であれば $n=3$，60% で $n=4$，95%では$n=39$ になる．

これは正規分布の性質に基づいている．その現象が正規分布に従うとすると，例えば信頼区間60%なら分布の中央60%の区間内にいると見なせる．
つまり，60%信頼区間の下限は「まだ寿命の20%」，上限は「もう寿命の80%」になる．このことから信頼区間$P$に対して，$n$の値は

$$n = \frac{2}{1 - P} - 1$$

で計算できる．40分降り続いている雨は，60%の確率であと10分～160分降り続く，ということだ．